# Handwritten Digit Classfication using Recurrent Neural Network

In this example, we are going to use the MNIST dataset to train a multi-layer feed foward neural network. MNIST is a simple computer vision dataset of handwritten digits. It has 60,000 training examles and 10,000 test examples. "It is a good database for people who want to try learning techniques and pattern recognition methods on real-world data while spending minimal efforts on preprocessing and formatting." For more details, please checkout the website [MNIST](http://yann.lecun.com/exdb/mnist/)

In [1]:
import org.apache.log4j.{Level, Logger}
import org.apache.spark.SparkContext

import com.intel.analytics.bigdl.utils._
import com.intel.analytics.bigdl.utils.{Engine, LoggerFilter, T, Table}
import com.intel.analytics.bigdl.dataset.DataSet
import com.intel.analytics.bigdl.dataset.image.{BytesToGreyImg, GreyImgNormalizer, GreyImgToBatch, GreyImgToSample}
import com.intel.analytics.bigdl.models.lenet.Utils._
import com.intel.analytics.bigdl.nn._
import com.intel.analytics.bigdl.optim._
import com.intel.analytics.bigdl.optim.{Adam, Top1Accuracy, Trigger}
import com.intel.analytics.bigdl.visualization.{TrainSummary, ValidationSummary}
import com.intel.analytics.bigdl.tensor.Tensor
import com.intel.analytics.bigdl.numeric.NumericFloat

Engine.init

## 1. Load MNIST dataset

In [2]:
import java.nio.ByteBuffer
import java.nio.file.{Files, Path, Paths}

import com.intel.analytics.bigdl.dataset.ByteRecord
import com.intel.analytics.bigdl.utils.File
import scopt.OptionParser

def load(featureFile: String, labelFile: String): Array[ByteRecord] = {
    val featureBuffer = ByteBuffer.wrap(Files.readAllBytes(Paths.get(featureFile)))
    val labelBuffer = ByteBuffer.wrap(Files.readAllBytes(Paths.get(labelFile)))
    
    val labelMagicNumber = labelBuffer.getInt()
    require(labelMagicNumber == 2049)
    val featureMagicNumber = featureBuffer.getInt()
    require(featureMagicNumber == 2051)

    val labelCount = labelBuffer.getInt()
    val featureCount = featureBuffer.getInt()
    require(labelCount == featureCount)

    val rowNum = featureBuffer.getInt()
    val colNum = featureBuffer.getInt()

    val result = new Array[ByteRecord](featureCount)
    var i = 0
    while (i < featureCount) {
      val img = new Array[Byte]((rowNum * colNum))
      var y = 0
      while (y < rowNum) {
        var x = 0
        while (x < colNum) {
          img(x + y * colNum) = featureBuffer.get()
          x += 1
        }
        y += 1
      }
      result(i) = ByteRecord(img, labelBuffer.get().toFloat + 1.0f)
      i += 1
    }

    result
}

Then we need to set paths of data. Please edit paths if they are changed.

In [3]:
val trainData = "../../datasets/mnist/train-images-idx3-ubyte"
val trainLabel = "../../datasets/mnist/train-labels-idx1-ubyte"
val validationData = "../../datasets/mnist/t10k-images-idx3-ubyte"
val validationLabel = "../../datasets/mnist/t10k-labels-idx1-ubyte"

## 2. Recurent Neural Network Model Setup

This time we will use a recurrent neural network (aka RNN) to classify handwritten digits. You can checkout this blog to get a detailed understanding of recurrent neural networks and LSTMs in particular.

In [4]:
//Parameters
val batchSize = 64
val maxEpochs = 5

//Network Parameters
val nInput = 28 //MNIST data input (img shape: 28*28)
val nHidden = 128 // hidden layer num of features
val nClasses = 10  //MNIST total classes (0-9 digits)

Then the data set should be created and the model needs to be established.

In [5]:
val trainSet = 
    DataSet.array(load(trainData, trainLabel), sc) -> BytesToGreyImg(28, 28) -> GreyImgNormalizer(trainMean, trainStd) -> GreyImgToBatch(batchSize)
val validationSet = 
    DataSet.array(load(validationData, validationLabel), sc) -> BytesToGreyImg(28, 28) -> GreyImgNormalizer(testMean, testStd) -> GreyImgToBatch(batchSize)

In [6]:
val recurrent = Recurrent().add(RnnCell(nInput, nHidden, Tanh()))
val rnnModel = Sequential().add(InferReshape(Array(-1, nInput), true)).add(recurrent).add(Select(2, -1)).add(Linear(nHidden, nClasses))

## 3. Optimizer Setup

In [7]:
val optimizer = Optimizer(model = rnnModel, dataset = trainSet, criterion = CrossEntropyCriterion[Float]())
optimizer.setValidation(trigger = Trigger.everyEpoch, dataset = validationSet, vMethods = Array(new Top1Accuracy))
optimizer.setOptimMethod(new Adam())
optimizer.setEndWhen(Trigger.maxEpoch(maxEpochs))

com.intel.analytics.bigdl.optim.DistriOptimizer@cecdb7c

The following is to create training and validation summary.

In [8]:
import java.text.SimpleDateFormat
import java.util.Calendar
val today = Calendar.getInstance
val formatDate = new SimpleDateFormat("yyyyMMdd-hhmmss")
val name = "rnn-" + formatDate.format(today.getTime()).toString()
val trainSummary = TrainSummary(logDir="/tmp/bigdl_summaries", appName=name)
trainSummary.setSummaryTrigger("Parameters", Trigger.severalIteration(50))
val valSummary = ValidationSummary(logDir="/tmp/bigdl_summaries", appName=name)
optimizer.setTrainSummary(trainSummary)
optimizer.setValidationSummary(valSummary)
printf("saving logs to %s", name)

saving logs to rnn-20170925-024042

In [9]:
// Boot training process
val trainedModel = optimizer.optimize()
print("Optimization Done.")

can't find locality partition for partition 0 Partition locations are (ArrayBuffer(172.168.2.109)) Candidate partition locations are
(0,List()).
Optimization Done.

In [10]:
val rddData = sc.parallelize(load(validationData, validationLabel), batchSize)
val transformer = BytesToGreyImg(28, 28) -> GreyImgNormalizer(testMean, testStd) -> GreyImgToSample()
val evaluationSet = transformer(rddData)
        
val result = trainedModel.evaluate(evaluationSet, Array(new Top1Accuracy[Float]), Some(batchSize))

result.foreach(r => println(s"${r._2} is ${r._1}"))

Top1Accuracy is Accuracy(correct: 9509, count: 10000, accuracy: 0.9509)


In [11]:
val predictions = trainedModel.predict(evaluationSet)
val preLabels = predictions.take(8).map(_.toTensor.max(1)._2.valueAt(1)).mkString(",")
val labels = evaluationSet.take(8).map(_.label.valueAt(1)).mkString(",")
println(preLabels)
println(labels)

5.0,2.0,5.0,10.0,5.0,2.0,5.0,10.0
8.0,3.0,2.0,1.0,5.0,2.0,5.0,10.0


In [12]:
val input = T.array(evaluationSet.take(8).map(_.feature))
val layer = JoinTable(2, 2)
val output = layer.forward(input)

In [13]:
import java.awt.Color
import java.awt.image.BufferedImage
import java.io.File
import javax.imageio

val imageSize = output.size()
val dim1 = imageSize(1)
val dim2 = imageSize(0)
println(dim1)
println(dim2)
val img = new BufferedImage(dim1, dim2, BufferedImage.TYPE_BYTE_GRAY)
for (i <- 0 until dim1)
    for (j <- 0 until dim2) {
        val value = (output.valueAt(j+1, i+1) + 0.5).toInt
        img.setRGB(i, j, new Color(value, value, value).getRGB)
    }
ImageIO.write(img, "PNG", new File("outimg.png")); 

224
28


Name: Compile Error
Message: <console>:80: error: not found: value ImageIO
       ImageIO.write(img, "PNG", new File("outimg.png"));
       ^
StackTrace: 

In [14]:
%%html
<body>
<img src="outimg.png"/>
</body>

## 4. Draw the performance curve

In [18]:
import vegas._
import vegas.render.HTMLRenderer._
import vegas.DSL._

In [21]:
val loss = trainSummary.readScalar("Loss")
val lossXY = loss.map(_ ._1).zip(loss.map(_ ._2)).toSeq
Vegas(description = "The Loss curve.", width = 700.0, height = 300.0).
  withXY(lossXY).
  encodeX("x", Quantitative, bin = Bin(maxbins = 500.0), title = "Iteration").
  encodeY("y", Quantitative, title = "Loss").
  mark(Line).
  show

<iframe id="frame-vegas-8fbf7795-4ceb-45cf-b9cb-5e08a8a2e145" sandbox="allow-scripts allow-same-origin" style="border: none; width: 100%" srcdoc="<html>
 <head>
 <script src="https://cdn.jsdelivr.net/webjars/org.webjars.bower/d3/3.5.17/d3.min.js" charset="utf-8"></script>
<script src="https://cdn.jsdelivr.net/webjars/org.webjars.bower/vega/2.6.3/vega.min.js" charset="utf-8"></script>
<script src="https://cdn.jsdelivr.net/webjars/org.webjars.bower/vega-lite/1.2.0/vega-lite.min.js" charset="utf-8"></script>
<script src="https://vega.github.io/vega-editor/vendor/vega-embed.js" charset="utf-8"></script>
 </head>
 <body>
 <div id='vegas-8fbf7795-4ceb-45cf-b9cb-5e08a8a2e145'></div>
 <script>
 var embedSpec = {
 mode: "vega-lite",
 spec: {
 "width" : 700.0,
 "height" : 300.0,
 "mark" : "line",
 "encoding" : {
 "x" : {
 "field" : "x",
 "type" : "quantitative",
 "bin" : {
 "maxbins" : 500.0
 },
 "title" : "Iteration"
 },
 "y" : {
 "field" : "y",
 "type" : "quantitative",
 "title" : "Loss"
 }
 },
 "description" : "The Loss curve.",
 "data" : {
 "values" : [
 {
 "x" : 1,
 "y" : 2.364739179611206
 },
 {
 "x" : 2,
 "y" : 2.3330960273742676
 },
 {
 "x" : 3,
 "y" : 2.274740219116211
 },
 {
 "x" : 4,
 "y" : 2.2809953689575195
 },
 {
 "x" : 5,
 "y" : 2.23933744430542
 },
 {
 "x" : 6,
 "y" : 2.26069974899292
 },
 {
 "x" : 7,
 "y" : 2.2187583446502686
 },
 {
 "x" : 8,
 "y" : 2.2401392459869385
 },
 {
 "x" : 9,
 "y" : 2.2304646968841553
 },
 {
 "x" : 10,
 "y" : 2.2320892810821533
 },
 {
 "x" : 11,
 "y" : 2.2386837005615234
 },
 {
 "x" : 12,
 "y" : 2.2125511169433594
 },
 {
 "x" : 13,
 "y" : 2.093998908996582
 },
 {
 "x" : 14,
 "y" : 2.1287012100219727
 },
 {
 "x" : 15,
 "y" : 2.106764316558838
 },
 {
 "x" : 16,
 "y" : 2.0175764560699463
 },
 {
 "x" : 17,
 "y" : 1.975579857826233
 },
 {
 "x" : 18,
 "y" : 1.969322919845581
 },
 {
 "x" : 19,
 "y" : 1.9882336854934692
 },
 {
 "x" : 20,
 "y" : 1.89456045627594
 },
 {
 "x" : 21,
 "y" : 1.9043813943862915
 },
 {
 "x" : 22,
 "y" : 1.8670395612716675
 },
 {
 "x" : 23,
 "y" : 1.8019574880599976
 },
 {
 "x" : 24,
 "y" : 1.8079320192337036
 },
 {
 "x" : 25,
 "y" : 1.8678607940673828
 },
 {
 "x" : 26,
 "y" : 1.948486089706421
 },
 {
 "x" : 27,
 "y" : 1.884664535522461
 },
 {
 "x" : 28,
 "y" : 1.8603912591934204
 },
 {
 "x" : 29,
 "y" : 1.7698464393615723
 },
 {
 "x" : 30,
 "y" : 1.7460670471191406
 },
 {
 "x" : 31,
 "y" : 1.7864408493041992
 },
 {
 "x" : 32,
 "y" : 1.75821852684021
 },
 {
 "x" : 33,
 "y" : 1.733232855796814
 },
 {
 "x" : 34,
 "y" : 1.8738596439361572
 },
 {
 "x" : 35,
 "y" : 1.6969749927520752
 },
 {
 "x" : 36,
 "y" : 1.8132543563842773
 },
 {
 "x" : 37,
 "y" : 1.7841854095458984
 },
 {
 "x" : 38,
 "y" : 1.643874168395996
 },
 {
 "x" : 39,
 "y" : 1.632593035697937
 },
 {
 "x" : 40,
 "y" : 1.7478394508361816
 },
 {
 "x" : 41,
 "y" : 1.6291472911834717
 },
 {
 "x" : 42,
 "y" : 1.6293741464614868
 },
 {
 "x" : 43,
 "y" : 1.6344757080078125
 },
 {
 "x" : 44,
 "y" : 1.6026475429534912
 },
 {
 "x" : 45,
 "y" : 1.5526816844940186
 },
 {
 "x" : 46,
 "y" : 1.4680876731872559
 },
 {
 "x" : 47,
 "y" : 1.637319803237915
 },
 {
 "x" : 48,
 "y" : 1.4602304697036743
 },
 {
 "x" : 49,
 "y" : 2.0266990661621094
 },
 {
 "x" : 50,
 "y" : 1.523977518081665
 },
 {
 "x" : 51,
 "y" : 1.4906044006347656
 },
 {
 "x" : 52,
 "y" : 1.6155441999435425
 },
 {
 "x" : 53,
 "y" : 1.5930144786834717
 },
 {
 "x" : 54,
 "y" : 1.540378212928772
 },
 {
 "x" : 55,
 "y" : 1.4816725254058838
 },
 {
 "x" : 56,
 "y" : 1.4429309368133545
 },
 {
 "x" : 57,
 "y" : 1.5195047855377197
 },
 {
 "x" : 58,
 "y" : 1.5828444957733154
 },
 {
 "x" : 59,
 "y" : 1.4154375791549683
 },
 {
 "x" : 60,
 "y" : 1.5153839588165283
 },
 {
 "x" : 61,
 "y" : 1.582270622253418
 },
 {
 "x" : 62,
 "y" : 1.6284862756729126
 },
 {
 "x" : 63,
 "y" : 1.55857515335083
 },
 {
 "x" : 64,
 "y" : 1.379510760307312
 },
 {
 "x" : 65,
 "y" : 1.3875925540924072
 },
 {
 "x" : 66,
 "y" : 1.4716649055480957
 },
 {
 "x" : 67,
 "y" : 1.4698362350463867
 },
 {
 "x" : 68,
 "y" : 1.

In [22]:
val top1 = valSummary.readScalar("Top1Accuracy")
val top1XY = top1.map(_ ._1).zip(top1.map(_ ._2)).toSeq
Vegas(description = "The Top1Accuracy curve.", width = 700.0, height = 300.0).
  withXY(top1XY).
  encodeX("x", Quantitative, bin = Bin(maxbins = 500.0), title = "Iteration").
  encodeY("y", Quantitative, bin = Bin(base = 0.9), title = "Top1Accuracy").
  mark(Line).
  show

<iframe id="frame-vegas-4ec998eb-ed98-419a-8fb9-2b83d494f38b" sandbox="allow-scripts allow-same-origin" style="border: none; width: 100%" srcdoc="<html>
 <head>
 <script src="https://cdn.jsdelivr.net/webjars/org.webjars.bower/d3/3.5.17/d3.min.js" charset="utf-8"></script>
<script src="https://cdn.jsdelivr.net/webjars/org.webjars.bower/vega/2.6.3/vega.min.js" charset="utf-8"></script>
<script src="https://cdn.jsdelivr.net/webjars/org.webjars.bower/vega-lite/1.2.0/vega-lite.min.js" charset="utf-8"></script>
<script src="https://vega.github.io/vega-editor/vendor/vega-embed.js" charset="utf-8"></script>
 </head>
 <body>
 <div id='vegas-4ec998eb-ed98-419a-8fb9-2b83d494f38b'></div>
 <script>
 var embedSpec = {
 mode: "vega-lite",
 spec: {
 "width" : 700.0,
 "height" : 300.0,
 "mark" : "line",
 "encoding" : {
 "x" : {
 "field" : "x",
 "type" : "quantitative",
 "bin" : {
 "maxbins" : 500.0
 },
 "title" : "Iteration"
 },
 "y" : {
 "field" : "y",
 "type" : "quantitative",
 "bin" : {
 "base" : 0.9
 },
 "title" : "Top1Accuracy"
 }
 },
 "description" : "The Top1Accuracy curve.",
 "data" : {
 "values" : [
 {
 "x" : 938,
 "y" : 0.8704000115394592
 },
 {
 "x" : 1876,
 "y" : 0.9239000082015991
 },
 {
 "x" : 2814,
 "y" : 0.9358999729156494
 },
 {
 "x" : 3752,
 "y" : 0.9430999755859375
 },
 {
 "x" : 4690,
 "y" : 0.9484000205993652
 }
 ]
 }
}
 }
 vg.embed("#vegas-4ec998eb-ed98-419a-8fb9-2b83d494f38b", embedSpec, function(error, result) {});
 </script>

 </body>
</html>">

Finally, the Spark should be stopped.

In [20]:
sc.stop()